# Full revision analysis

## List of work to be performed:
1. Supplementary figure showing incorrect inform tumor segmentation and NKT cells 

    1.2. images of zero NK cells from inForm and remake Figure 3a
2. Supplementary figure with zoom-in images of individual cells from inForm and TME-Analyzer in an image with high disagreement 
3. Incorporation of DeepCell into TME-Analyzer 
4. Reorganize the MonteCarlo trials 
5. Supplementary table 6 with full ranking of parameters and discovery/validation p values if top n parameters were used instead. 
6. Generate forest plot for 4E 
7. Build classifier based on train-test split of discovery cohort. Try 16-47, 32-31, 47-16 splits. 
8. ? It is significant amount of work, but since both reviewers mention is, I can train a random forest on the original dataset and report it ? 

## 1.2. Find and show images of 0 NK cells from inForm with increasing CD56 numbers

In [1]:
import pandas as pd
import numpy as np
import os
import shutil

TME_analyzer_folder = "E:/Desktop_backup_20231031/Desktop/TME-Analyzer_the_manuscript/stardist_analysis_20201108/analysis_results/"
px_size = 0.4999

combined_frame5 = pd.read_csv("E:/Moved from E/TME-Analyzer_the_manuscript/20211129_TME-Analyzer/data_analysis/reproduction_of_old_analysis/20230607_Marcel/combined_full.xls",sep='\t').set_index('sample id')

absolute_disagreement = []
for i_index in combined_frame5.index:
    disagreement = 0
    for i in combined_frame5.keys():
        try:
            if ((('inform' in i) | ('inForm' in i)) & ('in All' in i) & ('number' in i) & ('CD56' in i)):
                x = combined_frame5.loc[i_index,i]
                y = combined_frame5.loc[i_index,i.replace('inForm','auto')]
                if x + y  > 0:
                    disagreement += np.abs(x-y)/(x+y)
                elif x + y == 0:
                    disagreement += 0
                else:
                    disagreement = np.nan
        except:
            disagreement = np.nan
    absolute_disagreement.append([i_index, disagreement])
absolute_disagreement = np.array(absolute_disagreement)
absolute_disagreement_sorted = absolute_disagreement[absolute_disagreement[:, 1].argsort()]
print(absolute_disagreement_sorted)
print(np.sum(absolute_disagreement_sorted[:,1] == '1.0'))

[['RS-91-15929 ign_[33824,13937]' '0']
 ['DH-91-06974 ign_[54851,17999]' '0']
 ['DH-91-06974 ign_[54598,19885]' '0']
 ...
 ['DH-88-01020_[59608,9470]' 'nan']
 ['DH-88-01020_[61775,12537]' 'nan']
 ['ZR-89-02083_[40141,12993]' 'nan']]
314


In [2]:
images_of_interest = absolute_disagreement[absolute_disagreement[:,1] == '1.0',0]
print(images_of_interest[0],int(combined_frame5.loc[images_of_interest[0],'number of CD56 in All found by auto analysis']),len(images_of_interest))

BW-89-03894 infl_[33007,18868] 0 314


define plot composite image

In [18]:
from matplotlib import pyplot as plt
import numpy as np
color_LUT = {"red": [1, 0, 0], "green": [0, 1, 0], "blue": [0, 0, 1], "cyan": [0, 1, 1], "magenta": [1, 0, 1],
             "yellow": [1, 1, 0], "white": [1, 1, 1], "gray": [0.5, 0.5, 0.5], "black": [0, 0, 0], "aquamarine": [0.5, 1, 0.83],
             "coral": [1, 0.5, 0.3], "crimson": [0.85, 0.1, 0.3], "gold": [1, 0.83, 0], "lavender": [0.9, 0.9, 0.98],
             "olive": [0.5, 0.5, 0], "orange": [1, 0.5, 0], "orchid": [0.85, 0.44, 0.84], "pink": [1, 0.75, 0.8], "teal": [0, 0.5, 0.5]}

def display_composite_image(im_raw, ch_colors, Cell_props, pheno_names, pheno_colors):
    plt.figure(figsize=(13.4,10.04))
    n_channels = len(ch_colors)
    im_2_display = np.zeros((im_raw.shape[0],
                                im_raw.shape[1], 3), dtype=np.float32)
    for i in range(n_channels):
        Color_pointers_temp = ch_colors[i]
        for j in range(3):
            im_temp = im_2_display[:, :, j]
            im2_add = (im_raw[:, :, i]/im_raw[:, :, i].max())
            im2_add = im2_add * color_LUT[Color_pointers_temp][j]
            im_temp = im_temp + im2_add
            im_2_display[:, :, j] = im_temp
    plt.imshow(im_2_display/2)
    for i in range(len(pheno_names)):
        pheno_name = pheno_names[i]
        if pheno_name == 'CD4':
            Cells_of_interest = [i for i in Cell_props.index if ('CD3+' in Cell_props.loc[i,'Phenotypes']) & ('CD8+' not in Cell_props.loc[i,'Phenotypes'])]
        elif pheno_name == 'CD8':
            Cells_of_interest = [i for i in Cell_props.index if ('CD3+' in Cell_props.loc[i,'Phenotypes']) & ('CD8+' in Cell_props.loc[i,'Phenotypes'])]
        elif pheno_name == 'other':
            Cells_of_interest = [i for i in Cell_props.index if (['All'] == Cell_props.loc[i,'Phenotypes'])]
        else:
            Cells_of_interest = [i for i in Cell_props.index if ((pheno_name + '+') in Cell_props.loc[i,'Phenotypes'])]
        cell_locs = Cell_props.loc[Cells_of_interest,'Cell Centroid'].to_numpy()
        y =[j[0] for j in cell_locs]
        x =[j[1] for j in cell_locs]
        plt.scatter(x,y,s=3,marker='o',color=pheno_colors[i])
    plt.axis('off')
    
            


now find the images with zero CD56 with inform, and copy phenotype images with the number of CD56 cells found with TME-analyzer leading the filename

In [19]:
import pickle


workplace_folder = "E:/Moved from E/TME-Analyzer_the_manuscript/20211129_TME-Analyzer/workplaces/"
workplaces = [i for i in os.listdir(workplace_folder)]
inForm_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/Analysis Effector TNBC/image_with_phenotype_map/"
composite_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task1/inForm_output_images/"
inform_images = [i for i in os.listdir(inForm_folder)]
output_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task1/CD56_images/"
# pickle_data = pickle.load(open(workplace_folder + workplaces[0], "rb"))
for arg_index in images_of_interest:
    n_i = int(combined_frame5.loc[arg_index,'number of CD56 in All found by auto analysis'])
    if n_i == 0:
        n_i = int(combined_frame5.loc[arg_index,'number of CD56 in All found by inForm analysis'] * -1)
    i = [i for i in workplaces if arg_index in i][0]
    pickle_data = pickle.load(open(workplace_folder + i, "rb"))
    display_composite_image(pickle_data['im_raw'],pickle_data['Color_pointers'][:7],
                            pickle_data['Cell_props'],['CK','CD20','CD8','CD4','CD68','CD56','other'],
                            ['cyan','yellow','red','lime','orange','magenta','blue'])
    extent = plt.gca().get_window_extent().transformed(plt.gcf().dpi_scale_trans.inverted())
    plt.savefig(output_folder + f"{n_i:04d}_" + i + '.png', bbox_inches=extent)
    plt.close()
    color_pointers_mod = [i for i in pickle_data['Color_pointers'][:7]]
    for c_i, c in enumerate(color_pointers_mod):
        if c not in ['blue','magenta']:
            color_pointers_mod[c_i] = 'black'
    display_composite_image(pickle_data['im_raw'],color_pointers_mod,
                            pickle_data['Cell_props'],['CD56'],
                            ['magenta'])
    extent = plt.gca().get_window_extent().transformed(plt.gcf().dpi_scale_trans.inverted())
    plt.savefig(output_folder + f"{n_i:04d}_" + i + '_CD56.png', bbox_inches=extent)
    plt.close()
    i = [i for i in inform_images if arg_index in i][0]
    shutil.copy(inForm_folder + i,output_folder + f"{n_i:04d}_" + i)
    i = i.replace('_image_with_phenotype_map','_composite_image')
    shutil.copy(composite_folder + i, output_folder + f"{n_i:04d}_" + i)
    

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Now to remake figure 3A

In [3]:

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats


px_size = 0.4999

combined_frame5 = pd.read_csv("E:/Moved from E/TME-Analyzer_the_manuscript/20211129_TME-Analyzer/data_analysis/reproduction_of_old_analysis/20230607_Marcel/combined_full.xls",sep='\t').set_index('sample id')


output_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task1/"

for i in combined_frame5.keys():
    if (('inform' in i) | ('inForm' in i)) & ('Stroma' in i) & ('area' in i):
        mod_mult = (px_size**2) / 1000000
        label_add = ' ($mm^{2}$)'
        x = combined_frame5[i] * mod_mult
        y = combined_frame5[i.replace('inForm','auto')] * mod_mult
        y = y[x==x]
        x = x[x==x]
        x = x[y==y]
        y = y[y==y]
        x = x.fillna(0)
        y = y.fillna(0)
        slope, intercept, r, p, stderr = stats.linregress(x, y)
        # line = f'Stroma regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
        RMSE_percent = np.mean((x-y)**2)**0.5/(np.max(x)-np.min(x))*100
        if p> 0.0001:
            line = f'R={r:.2f}, p={p:.4f}, %RMSE={RMSE_percent:.0f}'
        else:
            line = f'R={r:.2f}, p<0.0001, %RMSE={RMSE_percent:.0f}'
        fig, ax = plt.subplots()
        fig.set_size_inches([5.195,4.78])
        ax.plot(x, y, linewidth=0, marker='s', label=i[:-25] + label_add, alpha = 0.5)
        ax.plot(x, intercept + slope * x, label=line)
        print(i,len(y))
        i = i.replace('Stroma','Tumor')
        x = combined_frame5[i] * mod_mult
        y = combined_frame5[i.replace('inForm','auto')] * mod_mult
        y = y[x==x]
        x = x[x==x]
        x = x[y==y]
        y = y[y==y]
        print(i,len(y))
        x = x.fillna(0)
        y = y.fillna(0)
        slope, intercept, r, p, stderr = stats.linregress(x, y)
        RMSE_percent = np.mean((x-y)**2)**0.5/(np.max(x)-np.min(x))*100
        # line = f'Tumor regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
        if p> 0.0001:
            line = f'R={r:.2f}, p={p:.4f}, %RMSE={RMSE_percent:.0f}'
        else:
            line = f'R={r:.2f}, p<0.0001, %RMSE={RMSE_percent:.0f}'
        ax.plot(x, y, linewidth=0, marker='o', label=i[:-25]+ label_add, alpha = 0.5)
        ax.plot(x, intercept + slope * x, label=line)
        ax.set_xlabel('InForm', fontsize=16)
        ax.set_ylabel('TME-Analyzer', fontsize=16)
        # ax.set_xlabel('', fontsize=16)
        # ax.set_ylabel('', fontsize=16)
        ax.legend(facecolor='white')
        ax.axis('square')
        plt.tight_layout()
        plt.yticks([ax.get_xticks()[1],ax.get_xticks()[-1]/2,ax.get_xticks()[-1]],fontsize=16)
        plt.xticks([ax.get_xticks()[1],ax.get_xticks()[-1]/2,ax.get_xticks()[-1]],fontsize=16)
        plt.savefig(output_folder + '' + i[:-34] + '.png')
        plt.close('all')

# for i in combined_frame5.keys():
#     if (('inform' in i) | ('inForm' in i)) & ('in All' in i):
#         if 'distance' in i:
#             x = combined_frame5[i]
#             y = combined_frame5[i[7:]]
#             y = y[x==x]
#             x = x[x==x]
#             x = x[y==y]
#             y = y[y==y]
#             print(i,len(y))
#             # x = x.fillna(0)
#             # y = y.fillna(0)
#             slope, intercept, r, p, stderr = stats.linregress(x, y)
#             # line = f'Regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
#             RMSE_percent = np.mean((x-y)**2)**0.5/(np.max(x)-np.min(x))*100
#             if p> 0.0001:
#                 line = f'R={r:.2f}\np={p:.4f}\n%RMSE={RMSE_percent:.0f}\n'
#             else:
#                 line = f'R={r:.2f}\np<0.0001\n%RMSE={RMSE_percent:.0f}\n'
#             fig, ax = plt.subplots()
#             fig.set_size_inches([5.195,4.78])
#             i = i.replace('CD3 CD8','CD8').replace('CD3','CD4').replace('of All','of all cells').replace('from All','from all cells')
#             ax.plot(x, y, linewidth=0, marker='s', label=i[6:-7] + ' (\u03bcm)', alpha = 0.5)
#             text_output = i[7:-7] + ' (\u03bcm)\n' + line
#             text_output = line
#             ax.plot(x, intercept + slope * x, label=line)

#             # i = i[:-6] + 'Tumor'
#             # x = 1/combined_frame5[i]
#             # y = 1/combined_frame5[i[7:] + '+']
#             # x = x.fillna(0)
#             # y = y.fillna(0)
#             # slope, intercept, r, p, stderr = stats.linregress(x, y)
#             # line = f'Tumor regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
#             # ax.plot(x, y, linewidth=0, marker='o', label='1/' + i[6:], alpha = 0.5)
#             # ax.plot(x, intercept + slope * x, label=line)
#             ax.set_xlabel('InForm', fontsize=16)
#             ax.set_ylabel('TME-Analyzer', fontsize=16)
#             if i[-17:-14] != 'CD8':
#                 ax.set_xlabel('', fontsize=16)
#                 ax.set_ylabel('', fontsize=16)
#             ax.set_xlabel('', fontsize=16)
#             ax.set_ylabel('', fontsize=16)
#             # ax.legend(facecolor='white')
#             ax.axis('square')
#             plt.text(ax.get_xticks()[-1]*0,ax.get_xticks()[-1]*0.7,text_output,fontsize=18)
#             plt.yticks([ax.get_xticks()[1],ax.get_xticks()[-1]/2,ax.get_xticks()[-1]],fontsize=16)
#             plt.xticks([ax.get_xticks()[1],ax.get_xticks()[-1]/2,ax.get_xticks()[-1]],fontsize=16)
#             plt.tight_layout()
#             plt.savefig(output_folder + 'correlation_plots/' + i[7:] + '.png')
#             plt.close('all')
#         else:
#             if 'density' in i:
#                 mod_mult = 1 / (px_size**2)
#                 label_add = ' (cells/$mm^{2}$)'
#             elif 'area' in i:
#                 mod_mult = (px_size**2) / 1000000
#                 label_add = ' ($mm^{2}$)'
#             else:
#                 mod_mult = 1
#                 label_add = ''
#             x = combined_frame5[i] * mod_mult
#             y = combined_frame5[i.replace('inForm','auto')] * mod_mult
#             i = i.replace('CD3 CD8','CD8').replace('CD3','CD4').replace('of All','of all cells').replace('from All','from all cells')
#             y = y[x==x]
#             x = x[x==x]
#             x = x[y==y]
#             y = y[y==y]
#             x = x.fillna(0)
#             y = y.fillna(0)
#             print(i,len(y))
#             slope, intercept, r, p, stderr = stats.linregress(x, y)
#             RMSE_percent = np.mean((x-y)**2)**0.5/(np.max(x)-np.min(x))*100
#             # line = f'Regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
#             if p> 0.0001:
#                 line = f'R={r:.2f}\np={p:.4f}\n%RMSE={RMSE_percent:.0f}\n'
#             else:
#                 line = f'R={r:.2f}\np<0.0001\n%RMSE={RMSE_percent:.0f}\n'
#             fig, ax = plt.subplots()
#             fig.set_size_inches([5.195,4.78])
#             ax.plot(x, y, linewidth=0, marker='s', label=i[:-32] + label_add, alpha = 0.5)
#             text_output = i[:-32] + label_add + '\n' + line
#             text_output = line
#             ax.plot(x, intercept + slope * x, label=line)

#             # i = i.replace('Stroma','Tumor')
#             # x = combined_frame5[i]
#             # y = combined_frame5[i.replace('inForm','auto')]
#             # x = x.fillna(0)
#             # y = y.fillna(0)
#             # slope, intercept, r, p, stderr = stats.linregress(x, y)
#             # line = f'Tumor regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
#             # ax.plot(x, y, linewidth=0, marker='o', label=i[:-25], alpha = 0.5)
#             # ax.plot(x, intercept + slope * x, label=line)
#             ax.set_xlabel('InForm', fontsize=16)
#             ax.set_ylabel('TME-Analyzer', fontsize=16)
#             ax.set_xlabel('', fontsize=16)
#             ax.set_ylabel('', fontsize=16)
#             # ax.legend(facecolor='white')
#             ax.axis('square')
#             plt.text(ax.get_xticks()[-1]*0,ax.get_xticks()[-1]*0.7,text_output,fontsize=18)
#             plt.yticks([ax.get_xticks()[1],ax.get_xticks()[-1]/2,ax.get_xticks()[-1]],fontsize=16)
#             plt.xticks([ax.get_xticks()[1],ax.get_xticks()[-1]/2,ax.get_xticks()[-1]],fontsize=16)
#             plt.tight_layout()

#             plt.savefig(output_folder + 'correlation_plots/' + i[:-25] + '.png')
#             plt.close('all')


area of Stroma found by inForm analysis 995
area of Tumor found by inForm analysis 995


In [4]:
print(len(x))

995
